In [1]:
import numpy as np, random
def mutation(arr,prob,numswi):
    num = len(arr)
    mutarr = arr
    for i in range(0,num):
        dice = random.uniform(0, 1)
        if dice <= prob:
            if mutarr[i] == 0:
                mutarr[i] = 1
            else:
                mutarr[i] = 0                 
    for i in range(0,numswi):
        dice = random.uniform(0, 1)
        if dice <= prob:
            p1 = random.randint(0,num-1)
            p2 = random.randint(0,num-1)
            mutarr[p1], mutarr[p2] = mutarr[p2], mutarr[p1]   
    
    return np.array(mutarr)

In [2]:
import numpy as np
def reproduction(par1,par2,l,u):
    num = len(par1)
    child  = par2[0:l] + par1[l:u+1] + par2[u+1:num]  
#    child2 = par1[0:l] + par2[l:u+1] + par1[u+1:num]
    return np.array(child)
#    return np.array(child), np.array(child2)

# Original Version

In [288]:
def GIPSolver(A,b,c,size,time,ratio,prob,l,u):
    cons, var = A.shape
    sols = np.array([np.zeros(var),np.ones(var)])
    # initial population #######################################################
    for i in range(size):
        sol = np.random.choice([0, 1], size=(var,), p=[1/3, 2/3])
        sols = np.append(sols,[sol],axis=0)
    LHS = np.dot(A,np.transpose(sols))    
    for i in range(size):    
        LHS[:,i] = LHS[:,i] - np.transpose(b) 
    LHS = LHS*100
    LHS[LHS>0] = 0
   # penalizing infeasibility better ##########################################
#   for i in range(size): 
#       for j in range(cons): 
#           if LHS[i,j]<0: 
#               LHS[i,j] = LHS[i,j] - 100
   ############################################################################
    score = np.dot(c,np.transpose(sols)) - LHS.sum(axis=0)
    sortindex = np.argsort(score)
   ############################################################################
    for t in range(time):          
   # Reproduction Phase - Crossover ###########################################
        numitem, rep = 0, 1
        children = np.empty([np.int(size*ratio)+1, var])
        while numitem <= np.int(size*ratio): 
            p1 = random.randint(0,np.int(size*ratio)-1)
            if rep == 1:
                p2 = random.randint(0,np.int(size*ratio)-1)
                child = reproduction(sols[p1,:].tolist(),sols[p2,:].tolist(),l,u)
                rep = 2
            else:
                p3 = random.randint(np.int(size*ratio),size-1)
                child = reproduction(sols[p1,:].tolist(),sols[p3,:].tolist(),l,u)
                rep = 1
            children[numitem,:] = child 
#           np.row_stack((children,child))
            numitem += 1
       
        sols = np.delete(sols,sortindex[np.int(size*0.5+1):np.int(size+2)],0)
        sols = np.row_stack((sols,children))
        # recalculation ########################################################
        LHS = np.dot(A,np.transpose(sols))    
        for i in range(LHS.shape[1]):    
            LHS[:,i] = LHS[:,i] - np.transpose(b) 
        LHS = LHS*100
        LHS[LHS>0] = 0
        score = np.dot(c,np.transpose(sols)) - LHS.sum(axis=0)
        sortindex = np.argsort(score)

        # Mutation Phase #######################################################
        nmut = 0
        while nmut < np.int(size*ratio):
            ind = random.randint(1,size-2)
            sols[sortindex[ind],:] = mutation(sols[sortindex[ind],:],prob,np.int(size*ratio))
            nmut += 1           
        # Migration Phase ######################################################
        sols = np.unique(sols, axis=0)
        distinct, var = np.shape(sols)
        if distinct < size+2:
            for i in range(size-distinct+2):
                sol = np.random.choice([0, 1], size=(var,), p=[1/3, 2/3])
                sols = np.append(sols,[sol],axis=0) 
        # recalculation        
        LHS = np.dot(A,np.transpose(sols))    
        for i in range(size+2):    
            LHS[:,i] = LHS[:,i] - np.transpose(b) 
        LHS = LHS*100
        LHS[LHS>0] = 0
        score = np.dot(c,np.transpose(sols)) - LHS.sum(axis=0)
        sortindex = np.argsort(score)

    return sols[sortindex[0],:], sum(sols[sortindex[0],:]*c)


# PART A

In [295]:
import numpy as np, random
from array import *

def GIPSolver_v2(A,b,c,size,time,ratio,prob,l,u):
    cons, var = A.shape
    sols = np.array([np.zeros(var),np.ones(var)])
    # initial population #######################################################
    for i in range(size):
        sol = np.random.choice([0, 1], size=(var,), p=[1/3, 2/3])
        sols = np.append(sols,[sol],axis=0)
    LHS = np.dot(A,np.transpose(sols))   
    for i in range(size):    
        LHS[:,i] = LHS[:,i] - np.transpose(b) 
    LHS = LHS*100
    LHS[LHS>0] = 0
    # penalizing infeasibility better ##########################################
    #   for i in range(size): 
    #       for j in range(cons): 
    #           if LHS[i,j]<0: 
    #               LHS[i,j] = LHS[i,j] - 100
    ############################################################################
    score = np.dot(c,np.transpose(sols)) - LHS.sum(axis=0)
    sortindex = np.argsort(score)
    ############################################################################
    for t in range(time):          
    # Reproduction Phase - Crossover ###########################################
        # Reproduction Phase - Crossover ###########################################
        numitem, rep = 0, 1
        #Sols matrix was used in unsorted shape, however in genetic reproduction it is necessary to identify strong members.
        #With using sortindex, our populations reproduction is far more efficient now.
        children = np.empty([np.int(size*ratio)+1, var])
        while numitem <= np.int(size*ratio): 
            p1 = random.randint(0,np.int(size*ratio)-1)
            if rep == 1:
                p2 = random.randint(0,np.int(size*ratio)-1)
                child = reproduction(sols[sortindex[p1],:].tolist(),sols[sortindex[p2],:].tolist(),l,u)
                rep = 2
            else:
                p3 = random.randint(np.int(size*ratio),size-1)
                child = reproduction(sols[sortindex[p1],:].tolist(),sols[sortindex[p3],:].tolist(),l,u)
                rep = 1
            children[numitem,:] = child 
        #           np.row_stack((children,child))
            numitem += 1

            #In previous version, delete operation was performed first and append operation of new reproduced members were second.
            #That was causing the loss of valuable members. 
            #The solution was appending new members, making a new calculation and then deleting useless members.
            # recalculation ########################################################
            LHS = np.dot(A,np.transpose(sols))    
            for i in range(LHS.shape[1]):    
                LHS[:,i] = LHS[:,i] - np.transpose(b) 
            LHS = LHS*100
            LHS[LHS>0] = 0
            score = np.dot(c,np.transpose(sols)) - LHS.sum(axis=0)
            sortindex = np.argsort(score)

            sols = np.delete(sols,sortindex[np.int(size+2):],0)

            # recalculation ########################################################
            LHS = np.dot(A,np.transpose(sols))    
            for i in range(LHS.shape[1]):    
                LHS[:,i] = LHS[:,i] - np.transpose(b) 
            LHS = LHS*100
            LHS[LHS>0] = 0
            score = np.dot(c,np.transpose(sols)) - LHS.sum(axis=0)
            sortindex = np.argsort(score)

            # Mutation Phase #######################################################
            #In mutation phase, original members were replaced by newly produced members without knowing the new ones are better.
            #That was causing loss of valuable members. 
            #The solution was appending new mutated members, making a new calculation and then deleting useless members.
            nmut = 0
            while nmut < np.int(size*ratio):
                ind = random.randint(1,size-2)
                sols = np.row_stack((sols, mutation(sols[sortindex[ind],:],prob,np.int(size*ratio))))
                nmut += 1           
            # Migration Phase ######################################################
            sols = np.unique(sols, axis=0)
            distinct, var = np.shape(sols)
            if distinct < size+2:
                for i in range(size-distinct+2):
                    sol = np.random.choice([0, 1], size=(var,), p=[1/3, 2/3])
                    sols = np.append(sols,[sol],axis=0) 
            # recalculation        
            LHS = np.dot(A,np.transpose(sols))    
            for i in range(size+2):    
                LHS[:,i] = LHS[:,i] - np.transpose(b) 
            LHS = LHS*100
            LHS[LHS>0] = 0
            score = np.dot(c,np.transpose(sols)) - LHS.sum(axis=0)
            sortindex = np.argsort(score) 
    print(sols[sortindex[0],:])
    print(sum(sols[sortindex[0],:]*c))

In [270]:
def optimal_knapsack_solver(A,b,c):
    #Create a list of all possible solutions 
    import itertools
    lst = list(map(list, itertools.product([0, 1], repeat=len(c))))
    # Multiply constraints with all possible solutions
    std = np.dot(A,np.transpose(lst)) 

    #Calculate the differences between calculated solutions and constraints
    diff = np.transpose(std) - np.transpose(b)

    #Calculate and append objective functions value and find optimal solution
    sol_list = []
    for i in range(0,len(diff)):
        if (diff[i][0] > 0) |  (diff[i][1] > 0) |  (diff[i][2] > 0) :
            sol_list.append(0)
        else: 
            sol_list.append(sum(lst[i]*c))

    optimal_solution, optimal_value = lst[sol_list.index(max(sol_list))] , max(sol_list)  
    print('Optimal Solution is:')
    print(optimal_solution)
    print(optimal_value)
    
anumeric_knapsack_solver(A,b,c)

Optimal Solution is:
[0, 0, 1, 1, 1, 0, 0, 0, 1, 1]
Optimal Value is:
14


### Problem 1

In [271]:
c = np.array([1, 2, 3, 4, 2, 1, 3, 1, 2, 3])
A = np.array([[1, 2, 2, 1, 2, 1, 3, 2, 1, 2],[2, 2, 2, 1, 0, 2, 0, 3, 2, 1], [3, 1, 0, 1, 2, 0, 1, 2, 3, 0]])
b = [8, 9, 7] 

#Optimal Solution
optimal_knapsack_solver(A,b,c)

#Original GIPSolver
sol, score = GIPSolver(A,b,c,100,100,0.5,0.2,4,5)
print('Original GIPSolver')
print(sol)
print(score)
#Modified GIPSolver
print('Modified GIPSolver')
GIPSolver_v2(A,b,c,100,100,0.5,0.2,4,5)   

Optimal Solution is:
[0, 0, 1, 1, 1, 0, 0, 0, 1, 1]
14
Original GIPSolver
[1. 1. 0. 0. 1. 1. 0. 1. 0. 0.]
7.0
Modified GIPSolver
[1. 1. 0. 0. 1. 1. 0. 1. 0. 0.]
7.0


### Problem 2

In [272]:
import random
c = np.array([random.randint(0,4) for i in range(10)])
A = np.array([[random.randint(0,4) for i in range(10)], [random.randint(0,4) for i in range(10)],[random.randint(0,4) for i in range(10)]])
b = [random.randint(10,12) for i in range(3)]

#Optimal Solution
optimal_knapsack_solver(A,b,c)

#Original GIPSolver
sol, score = GIPSolver(A,b,c,100,100,0.5,0.4,4,7)
print('Original GIPSolver')
print(sol)
print(score)
#Modified GIPSolver
print('Modified GIPSolver')
GIPSolver_v2(A,b,c,100,100,0.5,0.4,4,7)

Optimal Solution is:
[1, 1, 0, 0, 0, 1, 0, 1, 1, 1]
19
Original GIPSolver
[0. 1. 1. 0. 1. 0. 1. 0. 0. 1.]
11.0
Modified GIPSolver
[0. 1. 1. 1. 1. 0. 1. 0. 0. 1.]
11.0


### Problem 3

In [296]:
c = np.array([random.randint(0,4) for i in range(10)])
A = np.array([[random.randint(0,4) for i in range(10)], [random.randint(0,4) for i in range(10)],[random.randint(0,4) for i in range(10)]])
b = [random.randint(11,13) for i in range(3)]

#Optimal Solution
optimal_knapsack_solver(A,b,c)

#Original GIPSolver
sol, score = GIPSolver(A,b,c,100,100,0.5,0.4,4,7)
print('Original GIPSolver')
print(sol)
print(score)
#Modified GIPSolver
print('Modified GIPSolver')
GIPSolver_v2(A,b,c,100,100,0.5,0.4,4,7)

Optimal Solution is:
[1, 0, 0, 0, 0, 1, 0, 1, 1, 1]
15
Original GIPSolver
[0. 1. 1. 0. 0. 0. 0. 1. 1. 1.]
9.0
Modified GIPSolver
[0. 1. 1. 0. 0. 0. 0. 1. 1. 1.]
9.0


### Problem 4

In [274]:
c = np.array([random.randint(0,4) for i in range(10)])
A = np.array([[random.randint(0,4) for i in range(10)], [random.randint(0,4) for i in range(10)],[random.randint(0,4) for i in range(10)]])
b = [random.randint(9,12) for i in range(3)]

#Optimal Solution
optimal_knapsack_solver(A,b,c)

#Original GIPSolver
sol, score = GIPSolver(A,b,c,100,100,0.5,0.5,4,7)
print('Original GIPSolver')
print(sol)
print(score)
#Modified GIPSolver
print('Modified GIPSolver')
GIPSolver_v2(A,b,c,100,100,0.5,0.5,4,7)

Optimal Solution is:
[0, 0, 0, 1, 1, 0, 1, 0, 0, 1]
12
Original GIPSolver
[1. 0. 0. 1. 0. 1. 0. 1. 0. 0.]
5.0
Modified GIPSolver
[1. 0. 0. 1. 0. 1. 0. 1. 0. 0.]
5.0


### Problem 5

In [294]:
c = np.array([random.randint(0,4) for i in range(10)])
A = np.array([[random.randint(0,4) for i in range(10)], [random.randint(0,4) for i in range(10)],[random.randint(0,4) for i in range(10)]])
b = [random.randint(13,15) for i in range(3)]


#Optimal Solution
optimal_knapsack_solver(A,b,c)

#Original GIPSolver
sol, score = GIPSolver(A,b,c,50,100,0.5,0.2,3,4)
print('Original GIPSolver')
print(sol)
print(score)
#Modified GIPSolver
print('Modified GIPSolver')
GIPSolver_v2(A,b,c,50,100,0.5,0.2,3,4)

Optimal Solution is:
[0, 0, 1, 1, 0, 0, 1, 0, 1, 1]
19
Original GIPSolver
[1. 0. 1. 1. 1. 1. 0. 1. 0. 0.]
13.0
Modified GIPSolver
[1. 0. 1. 1. 1. 1. 0. 1. 0. 0.]
13.0


# PART B

In [359]:
def GIPSolver_eq(A,b,c,size,time,ratio,prob,l,u):
    cons, var = A.shape
    sols = np.array([np.zeros(var),np.ones(var)])
    # initial population #######################################################
    for i in range(size):
        sol = np.random.choice([0, 1], size=(var,), p=[1/3, 2/3])
        sols = np.append(sols,[sol],axis=0)
    LHS = np.dot(A,np.transpose(sols))    
    for i in range(size):    
        LHS[:,i] = LHS[:,i] - np.transpose(b) 
    LHS = LHS*100
   # penalizing infeasibility better ##########################################
#   for i in range(size): 
#       for j in range(cons): 
#           if LHS[i,j]<0: 
#               LHS[i,j] = LHS[i,j] - 100
   ############################################################################
    score = np.dot(c,np.transpose(sols)) + np.absolute(LHS.sum(axis=0))
    sortindex = np.argsort(score)
   ############################################################################
    for t in range(time):          
   # Reproduction Phase - Crossover ###########################################
        numitem, rep = 0, 1
        children = np.empty([np.int(size*ratio)+1, var])
        while numitem <= np.int(size*ratio): 
            p1 = random.randint(0,np.int(size*ratio)-1)
            if rep == 1:
                p2 = random.randint(0,np.int(size*ratio)-1)
                child = reproduction(sols[p1,:].tolist(),sols[p2,:].tolist(),l,u)
                rep = 2
            else:
                p3 = random.randint(np.int(size*ratio),size-1)
                child = reproduction(sols[p1,:].tolist(),sols[p3,:].tolist(),l,u)
                rep = 1
            children[numitem,:] = child 
#           np.row_stack((children,child))
            numitem += 1
       
        sols = np.delete(sols,sortindex[np.int(size*0.5+1):np.int(size+2)],0)
        sols = np.row_stack((sols,children))
        # recalculation ########################################################
        LHS = np.dot(A,np.transpose(sols))    
        for i in range(LHS.shape[1]):    
            LHS[:,i] = LHS[:,i] - np.transpose(b) 
        LHS = LHS*100
        score = np.dot(c,np.transpose(sols)) + np.absolute(LHS.sum(axis=0))
        sortindex = np.argsort(score)

        # Mutation Phase #######################################################
        nmut = 0
        while nmut < np.int(size*ratio):
            ind = random.randint(1,size-2)
            sols[sortindex[ind],:] = mutation(sols[sortindex[ind],:],prob,np.int(size*ratio))
            nmut += 1           
        # Migration Phase ######################################################
        sols = np.unique(sols, axis=0)
        distinct, var = np.shape(sols)
        if distinct < size+2:
            for i in range(size-distinct+2):
                sol = np.random.choice([0, 1], size=(var,), p=[1/3, 2/3])
                sols = np.append(sols,[sol],axis=0) 
        # recalculation        
        LHS = np.dot(A,np.transpose(sols))    
        for i in range(size+2):    
            LHS[:,i] = LHS[:,i] - np.transpose(b) 
        LHS = LHS*100
        score = np.dot(c,np.transpose(sols)) + np.absolute(LHS.sum(axis=0))
        sortindex = np.argsort(score)

    return sols[sortindex[0],:], sum(sols[sortindex[0],:]*c)


In [461]:
def optimal_knapsack_solver_eq(A,b,c):
    #Create a list of all possible solutions 
    import itertools
    lst = list(map(list, itertools.product([0, 1], repeat=len(c))))
    # Multiply constraints with all possible solutions
    std = np.dot(A,np.transpose(lst)) 

    #Calculate the differences between calculated solutions and constraints
    diff = np.transpose(std) - np.transpose(b)

    #Calculate and append objective functions value and find optimal solution
    sol_list = []
    for i in range(0,len(diff)):
        if (diff[i][0] == 0) :
            sol_list.append(sum(lst[i]*c))
        else: 
            sol_list.append(0)

    optimal_solution, optimal_value = lst[sol_list.index(max(sol_list))] , max(sol_list)  
    print('Optimal Solution is:')
    print(optimal_solution)
    print(optimal_value)
    

### Problem 1

In [479]:
c = np.array([random.randint(0,4) for i in range(5)])
A = np.array([[random.randint(0,4) for i in range(5)]])
b = [random.randint(7,9) for i in range(1)]


#Optimal Solution
optimal_knapsack_solver_eq(A,b,c)

#Original GIPSolver
sol, score = GIPSolver_eq(A,b,c,50,100,0.5,0.2,3,4)
print('Modified Equality GIPSolver')
print(sol)
print(score)

Optimal Solution is:
[1, 1, 1, 1, 1]
16
Modified Equality GIPSolver
[1. 0. 1. 1. 1.]
12.0
